<a href="https://colab.research.google.com/github/gggggeun/study/blob/main/MNIST(practice1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 흑백 손글씨 숫자 이미지 (MNIST) - practice1

### 손글씨 숫자 이미지 28x28픽셀 60,000개를 10개의 범주(0~9)로 분류
- numpy 배열 형태로 keras에 포함됨.
- 이미지와 레이블이 1:1 관계

In [1]:
# MNIST 데이터 불러오기
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [2]:
# train data
print("<train data - 28*28픽셀의 60,000개 이미지>\n")
print(train_images.shape)
print(len(train_labels))
print(train_labels)

print("\n--------------\n")

# test data
print("<test data - 28*28픽셀의 10,000개 이미지>\n")
print(test_images.shape)
print(len(test_labels))
print(test_labels)

<train data - 28*28픽셀의 60,000개 이미지>

(60000, 28, 28)
60000
[5 0 4 ... 5 6 8]

--------------

<test data - 28*28픽셀의 10,000개 이미지>

(10000, 28, 28)
10000
[7 2 1 ... 4 5 6]


## 작업 순서
0. 학습시킬 준비를 한다.(신경망 만들기, compile, 전처리)
1. train_images와 train_labels를 네트워크에 주입한다.
2. 네트워크는 이미지와 레이블을 연관시킬 수 있도록 학습한다.
3. test_images에 대한 예측을 네트워크에 요청하면 그 예측이 test_labels와 맞는지 확인할 것이다.

#### 학습 시킬 준비하기

In [3]:
# 간단한 신경망 만들기

from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28 ,)))
network.add(layers.Dense(10, activation='softmax'))

- **신경망의 핵심 구성 요소**는 일종의 데이터 처리 **필터**라고 생각할 수 있는 **층(layer)**이다. 이 층은 데이터가 입력되면 주어진 문제에 더 의미있는 **표현(representation)**을 추출한다.
- 위 코드의 예에서는 조밀하게 연결된 (또는 완전연결(**fully connected**)된) 신경망 층인 **Dense층** 2개가 연속되어 있다. 
- 두번째(마지막) 층은 10개의 확률 점수가 들어 있는 배열(모두 더하면 1)을 반환하는 **softmax** 층이다.
- 각 점수는 현재 숫자 이미지가 10개의 숫자 클래스 중 하나에 속할 확률이다.

In [4]:
# 신경망을 compile하는 단계

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

- 옵티마이저(optimizer) : 입력된 데이터와 손실 함수를 기반으로 네트워크를 업데이트하는 메커니즘
- 손실 함수(loss function) : 훈련 데이터에서 신경망의 성능을 측정하는 방법으로 네트워크가 옳은 방향으로 학습될 수 있도록 도와줌.
- 지표 : 훈련과 테스트 과정을 모니터링 할 수 있다. 여기에서는 accurac 지표를 사용함.

In [5]:
train_images.shape

(60000, 28, 28)

In [6]:
# 데이터 전처리
from keras.utils import to_categorical

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [7]:
train_images.shape

(60000, 784)

- 훈련을 하려면 데이터를 네트워크에 주입해야 하기 때문에, 데이터를 네트워크에 맞는 크기로 바꾼다. (reshape)
- 모든 값을 0과 1 사이로 스케일 조정한다. 현재 훈련 이미지는 [0,255] 사이의 값인 uint8 타입의 (60000, 28, 28) 크기를 가진 배열로 저장되어 있기 때문에 255로 나누면 모든 값이 0~1 값이 된다.
- 레이블을 범주형으로 인코딩한다. (to_categorical)

#### train data를 네트워크에 주입해 학습시킨다.

In [8]:
# 학습(fit)

network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 5s 9ms/step - loss: 0.4232 - accuracy: 0.8777
Epoch 2/5
469/469 [==============================] - 4s 9ms/step - loss: 0.1109 - accuracy: 0.9671
Epoch 3/5
469/469 [==============================] - 4s 9ms/step - loss: 0.0687 - accuracy: 0.9797
Epoch 4/5
469/469 [==============================] - 4s 9ms/step - loss: 0.0477 - accuracy: 0.9856
Epoch 5/5
469/469 [==============================] - 4s 9ms/step - loss: 0.0351 - accuracy: 0.9896


- loss, accuracy : 훈련 데이터에 대한 네트워크 손실과 정확도
- 훈련 데이터에 대해 정확도 98.96% 달성

#### test data를 예측하고 평가한다.

In [9]:
# 예측(evaluate)

test_loss, test_acc = network.evaluate(test_images, test_labels)
print('test_acc : ', test_acc)

313/313 [==============================] - 1s 2ms/step - loss: 0.0646 - accuracy: 0.9799
test_acc :  0.9799000024795532


- 테스트 데이터에 대한 정확도 97.99%
- 훈련 정확도보다 테스트 정확도가 좀 더 낮은데 과대적합(overfitting)이 일어났기 때문이다. 